In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd
from pyspark.sql.functions import col, expr
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean
from pyspark.sql.functions import lit
from pyspark.sql.functions import regexp_extract, regexp_replace
from pyspark.sql.functions import when
from pyspark.sql.functions import asc, desc
from pyspark.sql.functions import month, year, quarter

spark = SparkSession.builder.getOrCreate()

# 1.) Read the case, department, and source data into their own spark dataframes.

Let's see how writing to the local disk works in spark:

- Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
- Inspect your folder structure. What do you notice?
- Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [2]:
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType(
    [
        StructField("source_id", StringType()),
        StructField("source_username", StringType()),
    ]
)

schema1 = StructType(
    [
        StructField("case_id", StringType()),
        StructField("case_opened_date", StringType()),
        StructField("case_closed_date", StringType()),
        StructField("SLA_due_date", StringType()),
        StructField("case_late", StringType()),
        StructField("num_days_late", StringType()),
        StructField("case_closed", StringType()),
        StructField("dept_division", StringType()),
        StructField("service_request_type", StringType()),
        StructField("SLA_days", StringType()),
        StructField("case_status", StringType()),
        StructField("source_id", StringType()),
        StructField("request_address", StringType()),
        StructField("council_district", StringType()),
        
    ]
)

schema2 = StructType(
    [
        StructField("dept_division", StringType()),
        StructField("dept_name", StringType()),
        StructField("standardized_dept_name", StringType()),
        StructField("dept_subject_to_SLA", StringType())
    ]
)


# Read csv, but now we specify the schema:

source = spark.read.csv("source.csv", header=True, schema=schema)
case = spark.read.csv("case.csv", header=True, schema=schema1)
dept = spark.read.csv("dept.csv", header=True, schema=schema2)

In [3]:
# write data to a destination using .write property

source.write.json("source_json", mode="overwrite")
case.write.json("case_json", mode="overwrite")
dept.write.json("dept_json", mode="overwrite")

In [4]:
dept.show(1)

+---------------+----------------+----------------------+-------------------+
|  dept_division|       dept_name|standardized_dept_name|dept_subject_to_SLA|
+---------------+----------------+----------------------+-------------------+
|311 Call Center|Customer Service|      Customer Service|                YES|
+---------------+----------------+----------------------+-------------------+
only showing top 1 row



In [5]:
source.show()

+---------+--------------------+
|source_id|     source_username|
+---------+--------------------+
|   100137|    Merlene Blodgett|
|   103582|         Carmen Cura|
|   106463|     Richard Sanchez|
|   119403|      Betty De Hoyos|
|   119555|      Socorro Quiara|
|   119868| Michelle San Miguel|
|   120752|      Eva T. Kleiber|
|   124405|           Lori Lara|
|   132408|       Leonard Silva|
|   135723|        Amy Cardenas|
|   136202|    Michelle Urrutia|
|   136979|      Leticia Garcia|
|   137943|    Pamela K. Baccus|
|   138605|        Marisa Ozuna|
|   138650|      Kimberly Green|
|   138650|Kimberly Green-Woods|
|   138793| Guadalupe Rodriguez|
|   138810|       Tawona Martin|
|   139342|     Jessica Mendoza|
|   139344|        Isis Mendoza|
+---------+--------------------+
only showing top 20 rows



In [6]:
case.show(2, False)

+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+----------------------+-----------+-----------+---------+------------------------------------+----------------+
|case_id   |case_opened_date|case_closed_date|SLA_due_date|case_late|num_days_late      |case_closed|dept_division   |service_request_type  |SLA_days   |case_status|source_id|request_address                     |council_district|
+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+----------------------+-----------+-----------+---------+------------------------------------+----------------+
|1014127332|1/1/18 0:42     |1/1/18 12:29    |9/26/20 0:42|NO       |-998.5087616000001 |YES        |Field Operations|Stray Animal          |999.0      |Closed     |svcCRMLS |2315  EL PASO ST, San Antonio, 78207|5               |
|1014127333|1/1/18 0:46     |1/3/18 8:11     |1/5/18 8:30 |NO       |-2.01260416

### prepare:

In [7]:
df = case.join(dept, on=dept.dept_division == case.dept_division).drop(dept.dept_division)
df = df.join(source, on = df.source_id== source.source_id).drop(source.source_id)
df.printSchema()

root
 |-- case_id: string (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: string (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: string (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)
 |-- source_username: string (nullable = true)



In [8]:
df = df.withColumnRenamed("SLA_due_date", "case_due_date")

In [9]:
df = df.withColumn("case_closed", expr('case_closed == "YES"')).withColumn(
    "case_late", expr('case_late == "YES"'))
df = df.withColumn("council_district", col("council_district").cast("string"))

In [10]:
fmt = "M/d/yy H:mm"

df = df.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt)).withColumn("case_closed_date", to_timestamp("case_opened_date", fmt)).withColumn("case_due_date", to_timestamp("case_opened_date", fmt))

In [11]:
df = df.withColumn("request_address", trim(lower(df.request_address)))

In [12]:
df = df.withColumn(
    "num_weeks_late", expr("num_days_late / 7 AS num_weeks_late")
)


In [13]:
df = df.withColumn("council_district", col("council_district").cast("int"))

df = df.withColumn(
    "council_district",
    format_string("%03d", col("council_district").cast("int")),
)

In [14]:
df = df.withColumn("zipcode", regexp_extract("request_address", r"\d+$", 0))
df = (
    df.withColumn(
        "case_age", datediff(current_timestamp(), "case_opened_date")
    )
    .withColumn(
        "days_to_closed", datediff("case_closed_date", "case_opened_date")
    )
    .withColumn(
        "case_lifetime",
        when(expr("! case_closed"), col("case_age")).otherwise(
            col("days_to_closed")
        ),
    )
)

source.write.csv('source_csv', mode = 'overwrite')
case.write.csv('case_csv', mode = 'overwrite')
dept.write.csv('dept_csv', mode = 'overwrite')

In [15]:
df.dtypes

[('case_id', 'string'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('case_due_date', 'timestamp'),
 ('case_late', 'boolean'),
 ('num_days_late', 'string'),
 ('case_closed', 'boolean'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'string'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'string'),
 ('dept_name', 'string'),
 ('standardized_dept_name', 'string'),
 ('dept_subject_to_SLA', 'string'),
 ('source_username', 'string'),
 ('num_weeks_late', 'double'),
 ('zipcode', 'string'),
 ('case_age', 'int'),
 ('days_to_closed', 'int'),
 ('case_lifetime', 'int')]

#### The csv files contain a txt file which states succes if the csv was created, also the csv doesnt contain the column name but does contain two columns which has the id and username for source.

## 1.) How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?


In [32]:
df.sample(.1).show(10, vertical = True)

-RECORD 0--------------------------------------
 case_id                | 1014127361           
 case_opened_date       | 2018-01-01 07:13:00  
 case_closed_date       | 2018-01-01 07:13:00  
 case_due_date          | 2018-01-01 07:13:00  
 case_late              | false                
 num_days_late          | -14.70434028         
 case_closed            | true                 
 dept_division          | Code Enforcement     
 service_request_type   | Right Of Way/Side... 
 SLA_days               | 16.05280093          
 case_status            | Closed               
 source_id              | svcCRMSS             
 request_address        | 837  barrel point... 
 council_district       | 004                  
 dept_name              | Code Enforcement ... 
 standardized_dept_name | DSD/Code Enforcement 
 dept_subject_to_SLA    | YES                  
 source_username        | svcCRMSS             
 num_weeks_late         | -2.10062004          
 zipcode                | 78251         

In [17]:
df.groupby('case_status').agg(max("case_lifetime")).show()

+-----------+------------------+
|case_status|max(case_lifetime)|
+-----------+------------------+
|       Open|              1678|
|     Closed|                 0|
+-----------+------------------+



In [18]:
df.groupby('service_request_type').count().show(10)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|Minimum Housing-O...| 8617|
|        Tree Removal|  298|
| Service Information|  164|
|    Sign Maintenance|   82|
|Park Building Mai...|   48|
|Brush Property Da...|  186|
|Graffiti: Private...| 8525|
|Traffic Sign Graf...| 2150|
|License Renewal I...| 1394|
|Used/Scrap Tire F...|   19|
+--------------------+-----+
only showing top 10 rows



## 2.) How many Stray Animal cases are there?




In [19]:
df.filter(col('service_request_type') == 'Stray Animal').groupby('service_request_type').count().show()

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|        Stray Animal|27361|
+--------------------+-----+



## 3.) How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?


In [20]:
df.filter(col('dept_division') == 'Field Operations').filter(col('service_request_type') != 'Officer Standby').count()

116295

## 4.) Convert the council_district column to a string column.


In [21]:
#converted in prepare initial
df.dtypes

[('case_id', 'string'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('case_due_date', 'timestamp'),
 ('case_late', 'boolean'),
 ('num_days_late', 'string'),
 ('case_closed', 'boolean'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'string'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'string'),
 ('dept_name', 'string'),
 ('standardized_dept_name', 'string'),
 ('dept_subject_to_SLA', 'string'),
 ('source_username', 'string'),
 ('num_weeks_late', 'double'),
 ('zipcode', 'string'),
 ('case_age', 'int'),
 ('days_to_closed', 'int'),
 ('case_lifetime', 'int')]

## 5.) Extract the year from the case_closed_date column.


In [22]:
df1 = df.withColumn('year',year(df.case_closed_date))

## 6.) Convert num_days_late from days to hours in new columns num_hours_late.


In [23]:
df.dtypes

[('case_id', 'string'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('case_due_date', 'timestamp'),
 ('case_late', 'boolean'),
 ('num_days_late', 'string'),
 ('case_closed', 'boolean'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'string'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'string'),
 ('dept_name', 'string'),
 ('standardized_dept_name', 'string'),
 ('dept_subject_to_SLA', 'string'),
 ('source_username', 'string'),
 ('num_weeks_late', 'double'),
 ('zipcode', 'string'),
 ('case_age', 'int'),
 ('days_to_closed', 'int'),
 ('case_lifetime', 'int')]

In [24]:
#create new column which will have days to hours. 1 day = 24hrs.
df = df.withColumn('num_hours_late',expr("num_days_late / 24"))

## 6.) Join the case data with the source and department data.


did this in the prepare 

## 7.) Are there any cases that do not have a request source?

there is 0

In [25]:
df.filter(col('source_id') == lit('Null')).show()

+-------+----------------+----------------+-------------+---------+-------------+-----------+-------------+--------------------+--------+-----------+---------+---------------+----------------+---------+----------------------+-------------------+---------------+--------------+-------+--------+--------------+-------------+--------------+
|case_id|case_opened_date|case_closed_date|case_due_date|case_late|num_days_late|case_closed|dept_division|service_request_type|SLA_days|case_status|source_id|request_address|council_district|dept_name|standardized_dept_name|dept_subject_to_SLA|source_username|num_weeks_late|zipcode|case_age|days_to_closed|case_lifetime|num_hours_late|
+-------+----------------+----------------+-------------+---------+-------------+-----------+-------------+--------------------+--------+-----------+---------+---------------+----------------+---------+----------------------+-------------------+---------------+--------------+-------+--------+--------------+-------------+--

In [26]:
df.groupby('source_username').count().sort(asc("count")).show(20)

+-------------------+-----+
|    source_username|count|
+-------------------+-----+
|       Lexi Bachran|    1|
|     Chrissy McCain|    1|
|          Milee Ray|    1|
|    Sergio Gonzales|    1|
|   Emilie Christian|    1|
|      Janelle Lopez|    1|
|     Samantha Garza|    2|
|      Danette Brown|    2|
|     Erika Phillips|    2|
|       Jerry Robles|    2|
|        Andy Greene|    2|
|  Georgette Goacher|    2|
|       Linda Bibles|    3|
|        Raul Olveda|    3|
|   Jeremy Sifuentes|    3|
|       Sakib Shaikh|    5|
|      Vincent Kosub|    6|
| Elizabeth Stephens|    6|
|William F. Trynoski|    7|
|      Beau Anderson|    7|
+-------------------+-----+
only showing top 20 rows



## 8.) What are the top 10 service request types in terms of number of requests?


In [27]:
df.groupby('service_request_type').count().sort(desc("count")).show(10)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|           No Pickup|89210|
|Overgrown Yard/Trash|66403|
|        Bandit Signs|32968|
|        Damaged Cart|31163|
|Front Or Side Yar...|28920|
|        Stray Animal|27361|
|Aggressive Animal...|25492|
|Cart Exchange Req...|22608|
|Junk Vehicle On P...|21649|
|     Pot Hole Repair|20827|
+--------------------+-----+
only showing top 10 rows



## 9.) What are the top 10 service request types in terms of average days late?


In [28]:
df.groupby('service_request_type').agg(mean(col('num_days_late')))

DataFrame[service_request_type: string, avg(num_days_late): double]

In [29]:
df.groupby('service_request_type').agg(mean(col('num_days_late'))).sort(desc('avg(num_days_late)')).show(12)

+--------------------+------------------+
|service_request_type|avg(num_days_late)|
+--------------------+------------------+
|  Zoning: Junk Yards| 175.9563621042095|
|Labeling for Used...|162.43032902285717|
|Record Keeping of...|153.99724039428568|
|Signage Requied f...|151.63868055333333|
|Storage of Used M...|     142.112556415|
|Zoning: Recycle Yard|135.92851612479797|
|Donation Containe...|131.75610506358706|
|License Requied U...|128.79828704142858|
|Traffic Signal Gr...|101.79846062200002|
|           Complaint| 72.87050230311695|
|             Vendors|   66.548098985078|
|Reservation Assis...|       66.03116319|
+--------------------+------------------+
only showing top 12 rows



## Does number of days late depend on department?

yes certain departments are on average later than others

In [30]:
df.groupby('dept_division').agg(mean(col('num_days_late'))).sort(desc('avg(num_days_late)')).show(22)

+--------------------+-------------------+
|       dept_division| avg(num_days_late)|
+--------------------+-------------------+
|Code Enforcement ...| 135.92851612479797|
|        Reservations|        66.03116319|
|     311 Call Center| 59.737091496300785|
|Director's Office...| 37.570646702950086|
|Engineering Division|  13.43372455586971|
|               Shops|   9.64126176872269|
|           Tree Crew|  4.723282812065399|
|         Solid Waste| 3.5190239198762248|
|              Trades|  3.231977141276932|
|Clean and Green N...|  1.691468919487805|
|              Vector|-1.1206532993223441|
|    Facility License|-1.4126937702216633|
|       Miscellaneous|-1.7218576838926591|
|    Waste Collection| -2.170652238479914|
|     Clean and Green| -2.557154979254144|
|               Brush| -3.985790571457094|
| Food Establishments| -6.971552370451523|
|  Signs and Markings| -7.448628001357712|
|    Shops (Internal)| -8.186267119876439|
|         Storm Water|-14.055678397031881|
|Storm Wate

## How do number of days late depend on department and request type?

for a particular department division a particular request service type can result in a higher avg.

In [31]:
df.groupby('dept_division', 'service_request_type').agg(mean(col('num_days_late'))).sort(desc('avg(num_days_late)')).show(22)

+--------------------+--------------------+------------------+
|       dept_division|service_request_type|avg(num_days_late)|
+--------------------+--------------------+------------------+
|    Code Enforcement|  Zoning: Junk Yards| 175.9563621042095|
|Code Enforcement ...|Labeling for Used...|162.43032902285717|
|Code Enforcement ...|Record Keeping of...|153.99724039428568|
|Code Enforcement ...|Signage Requied f...|151.63868055333333|
|Code Enforcement ...|Storage of Used M...|     142.112556415|
|Code Enforcement ...|Zoning: Recycle Yard|135.92851612479797|
|    Code Enforcement|Donation Containe...|131.75610506358706|
|Code Enforcement ...|License Requied U...|128.79828704142858|
|             Signals|Traffic Signal Gr...|101.79846062200002|
|Code Enforcement ...|Used Mattress Ins...| 92.93570056835293|
|Code Enforcement ...|        Bandit Signs| 87.61764120320001|
|     311 Call Center|           Complaint| 72.87050230311695|
|    Code Enforcement|             Vendors|   66.548098